### Recommending books on the basis of total rating count

In [1]:
import pandas  as pd
import numpy as np


In [2]:
rating = pd.read_csv("BX-Book-Ratings.csv",sep = ";",encoding = 'latin-1',error_bad_lines = False)
books = pd.read_csv("BX-Books.csv",sep = ';',encoding = 'latin-1',error_bad_lines = False)
users = pd.read_csv("BX-Users.csv",sep = ';',encoding = 'latin-1',error_bad_lines = False)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\harsh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.p

In [3]:
rating.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [4]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [5]:
users.columns

Index(['User-ID', 'Location', 'Age'], dtype='object')

In [6]:
books_rating = pd.merge(rating,books,on= "ISBN")

In [7]:
#books_rating.drop(columns = ['Book-Author','Year-Of-Publication','Publisher','Image-URL-S', 'Image-URL-M', 'Image-URL-L'],inplace = True)

In [8]:
#books_rating.head()

In [9]:
# filtering user id having total count more than 200
# filtering book-rating having count grater than 100
count1 = rating["User-ID"].value_counts()
ratings = rating[rating["User-ID"].isin(count1[count1 > 200].index)]
counts = ratings["Book-Rating"].value_counts()
ratings = ratings[ratings["Book-Rating"].isin(counts[counts > 100].index)]
ratings.head()


,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [10]:
book_rating = pd.merge(books,ratings,on = "ISBN")
book_rating.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,85526,0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,96054,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,177458,0
4,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,110912,10


In [11]:
book_rating.drop(columns= ['Image-URL-S', 'Image-URL-M', 'Image-URL-L','Book-Author', 'Year-Of-Publication', 'Publisher'],inplace = True)

In [12]:
book_rating.head()


,ISBN,Book-Title,User-ID,Book-Rating
0,0002005018,Clara Callan,11676,8
1,0002005018,Clara Callan,85526,0
2,0002005018,Clara Callan,96054,0
3,0002005018,Clara Callan,177458,0
4,0374157065,Flu: The Story of the Great Influenza Pandemic...,110912,10


In [13]:
book_rating = book_rating.dropna(axis = 0,subset = ["Book-Title"])
book_rating.shape

(487671, 4)

In [14]:
# grouping book_rating by book title and book rating
book_ratings =(book_rating.groupby(by = ["Book-Title"])["Book-Rating"].count().reset_index().
                                    rename(columns = {"Book-Rating":"totalBookRating"})
                                    [["Book-Title","totalBookRating"]]
              )

In [15]:
book_ratings.head()

,Book-Title,totalBookRating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [16]:
book_rating_totalcount = pd.merge(book_rating,book_ratings,on = "Book-Title")
book_rating_totalcount.head()

,ISBN,Book-Title,User-ID,Book-Rating,totalBookRating
0,0002005018,Clara Callan,11676,8,4
1,0002005018,Clara Callan,85526,0,4
2,0002005018,Clara Callan,96054,0,4
3,0002005018,Clara Callan,177458,0,4
4,0374157065,Flu: The Story of the Great Influenza Pandemic...,110912,10,3


In [17]:
threshold = 50
book_rating_totalcount = book_rating_totalcount.query("totalBookRating > @threshold")
book_rating_totalcount.tail()


,ISBN,Book-Title,User-ID,Book-Rating,totalBookRating
247069,0743418131,Deck the Halls (Holiday Classics),250184,0,58
247070,0743418131,Deck the Halls (Holiday Classics),256407,0,58
247071,0743418131,Deck the Halls (Holiday Classics),261829,0,58
247072,0743418131,Deck the Halls (Holiday Classics),264031,0,58
247073,0743418131,Deck the Halls (Holiday Classics),266226,0,58


In [18]:
book_rating_totalcount.shape

(59903, 5)

In [19]:
from scipy.sparse import csr_matrix
book_rating_totalcount = book_rating_totalcount.drop_duplicates(["User-ID","Book-Title"])
book_rating_totalcount.head()


,ISBN,Book-Title,User-ID,Book-Rating,totalBookRating
7,0399135782,The Kitchen God's Wife,11676,9,111
8,0399135782,The Kitchen God's Wife,36836,0,111
9,0399135782,The Kitchen God's Wife,46398,9,111
10,0399135782,The Kitchen God's Wife,113270,0,111
11,0399135782,The Kitchen God's Wife,113519,0,111


In [20]:
book_rating_totalcount.shape

(57952, 5)

In [21]:
book_rating_totalcount_pivot = book_rating_totalcount.pivot(index = "Book-Title",columns = "User-ID",values = "totalBookRating").fillna(0)
book_rating_totalcount_pivot

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,76.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,124.0,0.0,0.0,0.0,0.0,124.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,0.0
A Bend in the Road,115.0,0.0,115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Case of Need,0.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,77.0,...,0.0,77.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,77.0
"A Child Called \It\"": One Child's Courage to Survive""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Civil Action,0.0,0.0,0.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
type(book_rating_totalcount_pivot)

pandas.core.frame.DataFrame

In [23]:
# converting the pivot table into sparse matrix 
book_rating_totalcount_matrix = csr_matrix(book_rating_totalcount_pivot.values)
book_rating_totalcount_matrix

<703x888 sparse matrix of type '<class 'numpy.float64'>'
	with 57952 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric = "cosine",algorithm = "brute")
model.fit(book_rating_totalcount_pivot)

NearestNeighbors(algorithm='brute', metric='cosine')

In [25]:
book_rating_totalcount_pivot.shape[0]

703

In [26]:
query_index = np.random.choice(book_rating_totalcount_pivot.shape[0])
print(query_index)

227


In [27]:
book_rating_totalcount_pivot.index[query_index]

'Harry Potter and the Prisoner of Azkaban (Book 3)'

In [28]:
distances,indices = model.kneighbors(book_rating_totalcount_pivot.iloc[query_index,:].values.reshape(1,-1),n_neighbors = 5)

In [29]:
for i in range(0,len(distances.flatten())):
    if i ==0:
        print("Recommendation for {0}:".format(book_rating_totalcount_pivot.index[query_index]))
        
    else:
        print("{0}: {1} with distance of :{2}".format(i,book_rating_totalcount_pivot.index[indices.flatten()[i]],distances.flatten()[i]))
        

Recommendation for Harry Potter and the Prisoner of Azkaban (Book 3):
1: Harry Potter and the Goblet of Fire (Book 4) with distance of :0.2865373291382419
2: Harry Potter and the Chamber of Secrets (Book 2) with distance of :0.3143784378973361
3: Harry Potter and the Order of the Phoenix (Book 5) with distance of :0.47534813684194666
4: Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)) with distance of :0.5256702610405394
